In [1]:
import util
import warnings
import random
import numpy as np
import torch
import pandas as pd

In [2]:
seed_val = 13
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [3]:
settings = util.parse_settings()
warnings.filterwarnings("ignore", category =UserWarning)

In [4]:
df = pd.read_parquet(settings['parquet'])
df.head(5)

,date,permno,GVKEY,CNAME,IBES_Timestamp,IBES_Actual_Date,Year,Quarter,rdq,SUE_Med,...,cik,filename,text,text_no_sw,text_stemmed,text_short,text_no_sw_lr_sent,text_short_lr_sent,text_no_sw_pr_sent,text_short_pr_sent
0,2003-01-03,19502,11264,WALGREEN CO,2003-01-03 08:55:00,2003-01-03,2003,1,2003-01-03,0.000346,...,1618921,None,None,None,None,None,NaN,NaN,NaN,NaN
1,2003-01-03,79255,28524,DELTA & PINE LND,2003-01-03 08:09:00,2003-01-03,2003,1,2003-01-03,-0.003868,...,902277,None,None,None,None,None,NaN,NaN,NaN,NaN
2,2003-01-07,69796,2710,CONSTELLAT BRAN,2003-01-06 16:31:00,2003-01-06,2003,1,2003-01-06,0.000213,...,16918,None,None,None,None,None,NaN,NaN,NaN,NaN
3,2003-01-07,76568,23697,SONIC CORP,2003-01-06 17:56:00,2003-01-06,2003,1,2003-01-06,0.000000,...,868611,None,None,None,None,None,NaN,NaN,NaN,NaN
4,2003-01-07,86822,119574,EXTREME NETWKS,2003-01-06 16:44:00,2003-01-06,2003,1,2003-01-06,-0.003003,...,1078271,86822_20030106_0.html,dated january extreme networks reports q fy fi...,dated january extreme networks reports financi...,date januari extrem network report financi res...,dated january extreme networks reports q fy fi...,0.012987,-0.01,0.01039,0.01


In [5]:
df = df.sort_values(by = settings['order'], ascending=True)
df = df[df[settings["text"]].notnull()][df[settings["target"]].notnull()]

model_name = "finbert"
#df = df[df['sp500'] == 1][df['Year']>=2003][df['Year']<=2006]
df = df[df['sp500'] == 1]
df['label'] = (df[settings["target"]] >=0).astype('int')
#df['label'] = df['sign']
df['text'] = df[settings["text"]]

In [6]:
df['label']

26        0
46        0
93        1
66        0
64        1
         ..
171159    0
171160    0
171140    0
171175    1
171170    1
Name: label, Length: 26788, dtype: int32

In [7]:
train_df = df.iloc[:int(df.shape[0]*0.7)]
test_df = df[int(df.shape[0]*0.7):]
train_df = train_df.reset_index()[['label', 'text']]
test_df = test_df.reset_index()[['label', 'text']]
train_df.shape, test_df.shape

((18751, 2), (8037, 2))

In [8]:
from datasets import Dataset
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


train_tokenized_datasets = train_ds.map(tokenize_function, batched=True)
test_tokenized_datasets = test_ds.map(tokenize_function, batched=True)

100%|██████████| 9/9 [00:16<00:00,  1.85s/ba]


In [40]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

loading configuration file https://huggingface.co/ProsusAI/finbert/resolve/main/config.json from cache at C:\Users\Steve/.cache\huggingface\transformers\2120f4f96b5830e5a91fe94d242471b0133b0976c8d6e081594ab837ac5f17bc.ef97278c578016c8bb785f15296476b12eae86423097fed78719d1c8197a3430
Model config BertConfig {
  "_name_or_path": "ProsusAI/finbert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "neutral": 2,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "ab

In [46]:
import torch.nn as nn
model.num_labels = 2 
model.classifier = nn.Linear(768,2)

In [47]:
model.classifier = model.classifier = nn.Sequential(
    nn.Linear(768, 512), 
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, 100),
    nn.ReLU(),
    nn.Linear(100, 2)
)

In [66]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 18751
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 46880
  0%|          | 0/46880 [00:26<?, ?it/s]

                                                   Saving model checkpoint to test_trainer_class_tuned_4\checkpoint-5000
Configuration saved in test_trainer_class_tuned_4\checkpoint-5000\config.json


{'loss': 0.6954, 'learning_rate': 8.93344709897611e-05, 'epoch': 0.53}


Model weights saved in test_trainer_class_tuned_4\checkpoint-5000\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8037
  Batch size = 2
100%|██████████| 4019/4019 [02:05<00:00, 31.83it/s]
                                                      


{'eval_loss': 0.692639172077179, 'eval_accuracy': 0.5164862510887147, 'eval_runtime': 125.7784, 'eval_samples_per_second': 63.898, 'eval_steps_per_second': 31.953, 'epoch': 1.0}



                                                    Saving model checkpoint to test_trainer_class_tuned_4\checkpoint-10000
Configuration saved in test_trainer_class_tuned_4\checkpoint-10000\config.json


{'loss': 0.694, 'learning_rate': 7.866894197952219e-05, 'epoch': 1.07}


Model weights saved in test_trainer_class_tuned_4\checkpoint-10000\pytorch_model.bin

                                                    Saving model checkpoint to test_trainer_class_tuned_4\checkpoint-15000
Configuration saved in test_trainer_class_tuned_4\checkpoint-15000\config.json


{'loss': 0.6936, 'learning_rate': 6.800341296928327e-05, 'epoch': 1.6}


Model weights saved in test_trainer_class_tuned_4\checkpoint-15000\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8037
  Batch size = 2
100%|█████████▉| 4017/4019 [01:37<00:00, 41.22it/s]
                                                     

{'eval_loss': 0.6930135488510132, 'eval_accuracy': 0.5164862510887147, 'eval_runtime': 97.7947, 'eval_samples_per_second': 82.182, 'eval_steps_per_second': 41.096, 'epoch': 2.0}


In [ ]:
trainer.predict(train_tokenized_datasets)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 18751
  Batch size = 2
100%|█████████▉| 9372/9376 [03:19<00:00, 46.93it/s]

PredictionOutput(predictions=array([[-0.004141  ,  0.08265184],
       [-0.0041481 ,  0.08266003],
       [-0.00415605,  0.08265426],
       ...,
       [-0.00414719,  0.08265813],
       [-0.00414813,  0.08265999],
       [-0.00415218,  0.08265587]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 1, 0], dtype=int64), metrics={'test_loss': 0.6928921341896057, 'test_accuracy': 0.5137859314169911, 'test_runtime': 199.8976, 'test_samples_per_second': 93.803, 'test_steps_per_second': 46.904})

100%|██████████| 9376/9376 [03:36<00:00, 46.93it/s]

In [29]:
train_tokenized_datasets = train_tokenized_datasets.shuffle(seed=42)
test_tokenized_datasets = test_tokenized_datasets.shuffle(seed=42)

In [58]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer_class_tuned_4", evaluation_strategy="epoch",per_device_train_batch_size=2, per_device_eval_batch_size=2, num_train_epochs=5, report_to="all",
                                logging_steps=5000, save_steps=5000, weight_decay=0.2, learning_rate=1e-4

)

PyTorch: setting up devices


In [43]:
from datasets import load_metric
#from sklearn.metrics import r2_score
metric = load_metric('accuracy')
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = np.argmax(logits, axis=1)
    return metric.compute(predictions=logits, references=labels)

In [54]:
from transformers import Trainer
trainer = Trainer(
     model=model,
     args=training_args,
     train_dataset=train_tokenized_datasets,
     eval_dataset=test_tokenized_datasets,
     compute_metrics=compute_metrics,
)

In [65]:
torch.cuda.empty_cache()
torch.cuda.memory_summary()
import gc
gc.collect()


128

In [22]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 18751
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 46880
  1%|          | 500/46880 [00:48<1:03:54, 12.09it/s]Saving model checkpoint to test_trainer_class_1\checkpoint-500
Configuration saved in test_trainer_class_1\checkpoint-500\config.json


{'loss': 0.6852, 'learning_rate': 4.946672354948806e-05, 'epoch': 0.05}


Model weights saved in test_trainer_class_1\checkpoint-500\pytorch_model.bin
  2%|▏         | 1000/46880 [01:38<1:15:28, 10.13it/s]Saving model checkpoint to test_trainer_class_1\checkpoint-1000
Configuration saved in test_trainer_class_1\checkpoint-1000\config.json


{'loss': 0.6954, 'learning_rate': 4.893344709897611e-05, 'epoch': 0.11}


Model weights saved in test_trainer_class_1\checkpoint-1000\pytorch_model.bin
  3%|▎         | 1500/46880 [02:23<1:04:27, 11.73it/s]Saving model checkpoint to test_trainer_class_1\checkpoint-1500
Configuration saved in test_trainer_class_1\checkpoint-1500\config.json


...

Model weights saved in test_trainer_class_1\checkpoint-46000\pytorch_model.bin
 99%|█████████▉| 46500/46880 [1:54:01<01:51,  3.40it/s]Saving model checkpoint to test_trainer_class_1\checkpoint-46500
Configuration saved in test_trainer_class_1\checkpoint-46500\config.json


{'loss': 0.6955, 'learning_rate': 4.052901023890785e-07, 'epoch': 4.96}


Model weights saved in test_trainer_class_1\checkpoint-46500\pytorch_model.bin
100%|██████████| 46880/46880 [1:56:00<00:00,  3.29it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8037
  Batch size = 2
                                                       
100%|██████████| 46880/46880 [1:58:45<00:00,  3.29it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 46880/46880 [1:58:45<00:00,  6.58it/s]

{'eval_loss': 0.6928670406341553, 'eval_accuracy': 0.5164862510887147, 'eval_runtime': 164.4131, 'eval_samples_per_second': 48.883, 'eval_steps_per_second': 24.445, 'epoch': 5.0}
{'train_runtime': 7125.4341, 'train_samples_per_second': 13.158, 'train_steps_per_second': 6.579, 'train_loss': 0.6958916348402003, 'epoch': 5.0}


TrainOutput(global_step=46880, training_loss=0.6958916348402003, metrics={'train_runtime': 7125.4341, 'train_samples_per_second': 13.158, 'train_steps_per_second': 6.579, 'train_loss': 0.6958916348402003, 'epoch': 5.0})

In [24]:
trainer.predict(train_tokenized_datasets)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 18751
  Batch size = 2
13391it [04:51, 51.09it/s]                         

PredictionOutput(predictions=array([[-0.03326268, -0.01324107],
       [-0.03327196, -0.01322575],
       [-0.03326493, -0.01325857],
       ...,
       [-0.03326451, -0.01323024],
       [-0.03326571, -0.01323332],
       [-0.03327554, -0.01322028]], dtype=float32), label_ids=array([1, 0, 1, ..., 0, 1, 0], dtype=int64), metrics={'test_loss': 0.6929210424423218, 'test_accuracy': 0.5137859314169911, 'test_runtime': 181.5791, 'test_samples_per_second': 103.266, 'test_steps_per_second': 51.636})

13395it [05:09, 51.09it/s]

In [23]:
trainer.predict(test_tokenized_datasets)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 8037
  Batch size = 2
100%|██████████| 4019/4019 [01:50<00:00, 53.43it/s]

PredictionOutput(predictions=array([[-0.03327847, -0.01322578],
       [-0.0332557 , -0.01324651],
       [-0.03325518, -0.01323215],
       ...,
       [-0.03328364, -0.01322294],
       [-0.03325091, -0.01324074],
       [-0.03327898, -0.01322959]], dtype=float32), label_ids=array([0, 1, 0, ..., 1, 1, 0], dtype=int64), metrics={'test_loss': 0.6928670406341553, 'test_accuracy': 0.5164862510887147, 'test_runtime': 110.4596, 'test_samples_per_second': 72.76, 'test_steps_per_second': 36.384})